In [1]:
!python -V

Python 3.10.6


In [2]:
import pandas as pd
import numpy as np

import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import mlflow

In [3]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

2023/05/28 18:28:28 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/05/28 18:28:28 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/home/moses/Documents/mlops-zoomcamp/week2/mlruns/1', creation_time=1685294911602, experiment_id='1', last_update_time=1685294911602, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
jan_df = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')

In [5]:
jan_df['duration'] = jan_df.tpep_dropoff_datetime - jan_df.tpep_pickup_datetime
jan_df.duration = jan_df.duration.apply(lambda td: td.total_seconds() / 60)

In [6]:
jan_df = jan_df[(jan_df.duration >= 1) & (jan_df.duration <= 60)]

In [7]:
categorical = ['PULocationID', 'DOLocationID']

jan_df[categorical] = jan_df[categorical].astype(str)

In [8]:
train_dicts = jan_df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [9]:
target = 'duration'
y_train = jan_df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [10]:
# Number 5
y_pred = lr.predict(X_train)

train_rmse=mean_squared_error(y_train, y_pred, squared=False)

In [11]:
print(train_rmse)

6.986191065500608


In [14]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [15]:
df_val = read_dataframe('data/yellow_tripdata_2022-02.parquet')

In [16]:
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

In [17]:
target = 'duration'
y_val = df_val[target].values

In [18]:
# Number 6
y_pred_val = lr.predict(X_val)

val_rmse=mean_squared_error(y_val, y_pred_val, squared=False)

In [19]:
print(val_rmse)

7.786408015215065


## Adding the mlflow functionality

In [ ]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "Moses Daudu")
    
    mlflow.log_param("train-data-path", "data/yellow_tripdata_2022-01.parquet")
    mlflow.log_param("val-data-path", "data/yellow_tripdata_2022-02.parquet")
    
    sample=np.arange(0,1,0.01)
    
    for samp in sample:
        alpha=samp
        mlflow.log_param("alpha", alpha)
    
        las = Lasso(alpha)
        las.fit(X_train, y_train)
    
        y_pred_train=las.predict(X_val)
        
        train_rmse=mean_squared_error(y_val, y_pred_train, squared=False)
        mlflow.log_metric("RMSE", train_rmse)
    
    

/tmp/ipykernel_6753/2952206573.py:15: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  las.fit(X_train, y_train)


In [24]:
sample= np.arange(0, 1, 0.01)

In [26]:
len(sample)

100

## Second Part of Mlflow

In [20]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("developer", "Moses Daudu")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=10,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}


In [24]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=25,
    trials=Trials()
)

[10:25:25] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:8.55580                                                     
[1]	validation-rmse:7.64329                                                     
[2]	validation-rmse:7.39879                                                     
[3]	validation-rmse:7.05263                                                     
[4]	validation-rmse:6.96653                                                     
[5]	validation-rmse:6.88865                                                     
[6]	validation-rmse:6.74579                                                     
[7]	validation-rmse:6.70115                                                     
[8]	validation-rmse:6.55471                                                     
[9]	validation-rmse:6.42374                                                     
[10:25:48] WARNING: ../src/objective/regression_obj.cu:213: reg:linear 

[4]	validation-rmse:10.65116                                                    
[5]	validation-rmse:10.02794                                                    
[6]	validation-rmse:9.38923                                                     
[7]	validation-rmse:8.96323                                                     
[8]	validation-rmse:8.50577                                                     
[9]	validation-rmse:8.21846                                                     
[10:37:11] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:7.45954                                                     
[1]	validation-rmse:6.43339                                                     
[2]	validation-rmse:6.00855                                                     
[3]	validation-rmse:5.91476                                                     
[4]	validation-rmse:5.72197                                            

In [ ]:
param = {
    'learning_rate' : 0.6620269443945731
    'max_depth' : 75
    'min_child_weight' : 0.6681121434468128
    'objective' : 'reg:linear'
    'reg_alpha' : 0.009835858863989126
    'reg_lambda' : 0.23315101108258487
    'seed' : 42
}

boster = fmin()